In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from wordcloud import WordCloud
from collections import Counter
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 종목 추천 시스템 개발

### 세가지 요소를 조합해서 추천

1. (재무적) ESG 점수 : 위에서 구한 ESG 점수에 선호 보유 기간 clustering한 결과에 따라 가중치를 곱해준 후 더해줌
- 단기(cluster2), 중기(cluster0), 중장기(cluster3), 장기(cluster1) 보유 선호 순으로 각각 ESG 점수에 0.25, 0.5, 0.75, 1을 곱해준 후 더해줌


2. (비재무적) 기업 가치 점수 : 위에서 구한 기업 가치 종합 점수를 더해줌

3. 선호 업종(중분류) : 앞에서 구한 선호 업종 탑3에 해당하는 업종들에게는 `총매수X위금액 / sum(총매수X위금액)` 만큼의 점수를 더해줌. 

In [ ]:
#데이터 준비하기
#미래에셋 데이터
mirae = pd.read_csv('/content/drive/MyDrive/미래에셋/mirae_stockname.csv', index_col = 0)
#업종(중분류, 소분류) 데이터
upjong_price = pd.read_csv('/content/drive/MyDrive/미래에셋/전처리/data/upjong_price.csv', index_col = 0)
#선호 보유 기간 cluster 데이터
df_jangdan = pd.read_csv('/content/drive/MyDrive/미래에셋/전처리/data/jangdan_cluster.csv')
#ESG 점수 데이터
df_ESG = pd.read_csv('/content/drive/MyDrive/미래에셋/ESG점수/data/ESG_final_norm.csv', index_col = 0)
#기업 가치 점수 데이터
df_stock_pointer = pd.read_csv('/content/drive/MyDrive/미래에셋/기업 가치 점수/stock_pointer.csv', index_col = 0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [ ]:
def stock_recommendation(idx):
  df_score = pd.merge(df_stock_pointer, df_ESG, how = 'left', left_on = '종목명', right_on = 'Stock')
  df_jangdan_rename = df_jangdan.rename(columns = {'Unnamed: 0': 'ID'})
  
  # ESG 점수를 선호 보유기간에 따라 조정
  if idx in list(df_jangdan_rename['ID']):
    if list(df_jangdan_rename[df_jangdan_rename['ID'] == idx]['jangdan_cluster'])[0] == 2: #단기
      df_score['Score_norm'] = df_score['Score_norm'] * 0.25
    elif list(df_jangdan_rename[df_jangdan_rename['ID'] == idx]['jangdan_cluster'])[0] == 0: #중기
      df_score['Score_norm'] = df_score['Score_norm'] * 0.5
    elif list(df_jangdan_rename[df_jangdan_rename['ID'] == idx]['jangdan_cluster'])[0] == 3: #중장기
      df_score['Score_norm'] = df_score['Score_norm'] * 0.75
    elif list(df_jangdan_rename[df_jangdan_rename['ID'] == idx]['jangdan_cluster'])[0] == 1: #장기
      df_score['Score_norm'] = df_score['Score_norm']
  else: #결측치일 경우
    df_score['Score_norm'] = 0
  
  # 총 점수를 구함
  df_score['TOTAL_SCORE'] = df_score['Score_norm'] + df_score['VALUE_SCORE']
  
  # 앞에서 구한 선호 업종 탑3에 해당하는 업종들에게는 `총매수X위금액 / sum(총매수X위금액)` 만큼의 점수를 더해줌. 
  sum_price = upjong_price['총매수1위금액'][idx] + upjong_price['총매수2위금액'][idx] + upjong_price['총매수3위금액'][idx]
  if upjong_price['총매수1위금액'][idx] != 0:
    df_score.loc[df_score[df_score['중분류'] == upjong_price['총매수1위업종'][idx]].index,'TOTAL_SCORE'] += upjong_price['총매수1위금액'][idx] / sum_price
  if upjong_price['총매수2위금액'][idx] != 0:
    df_score.loc[df_score[df_score['중분류'] == upjong_price['총매수2위업종'][idx]].index,'TOTAL_SCORE'] += upjong_price['총매수2위금액'][idx] / sum_price
  if upjong_price['총매수3위금액'][idx] != 0:
    df_score.loc[df_score[df_score['중분류'] == upjong_price['총매수3위업종'][idx]].index,'TOTAL_SCORE'] += upjong_price['총매수3위금액'][idx] / sum_price
  
  # 구해준 점수에 따라 높은 점수부터 정렬
  df_score = df_score.sort_values(by=['TOTAL_SCORE'], ascending= False)
  df_score.reset_index(drop = True, inplace = True)

  ### 위에서 구한 종목 순위 기반, 이제껏 매수했던 종목들과 겹치지 않는 종목을 추천해주기
  
  mirae_stock_list = []
  mirae_columns = list(mirae.columns)
  for x in mirae_columns:
    if '주식매수' in x:
      if len(x) > 13 and not '금액' in x:
        mirae_stock_list.append(x)

  # 이제껏 매수해온 종목들 구하기
  maesu_stock = []
  for s in mirae_stock_list:
    maesu_stock.append(mirae[s][idx])
  maesu_stock = list(set(maesu_stock))

  # 이제 순위에서, 이제껏 매수해온 종목이 아닌 종목들만 높은 순위부터 3개 추천해주기
  recommandation = []
  rank = 0
  while True:
    if df_score['종목명'][rank] not in maesu_stock:
      recommandation.append(df_score['종목명'][rank])
    rank += 1
    if len(recommandation) == 3:
      break
  
  return recommandation

In [ ]:
stock_recommendation(3)

['한진칼', '에스디바이오센서', '메리츠화재']